In [2]:
import os
from github import Github
from tqdm.autonotebooks import tqdm

gh = Github(os.environ['GITHUB_TOKEN'], per_page=100)
nixpkgs = gh.get_repo('nixos/nixpkgs')
all_prs = list(tqdm(nixpkgs.get_pulls(state="open")))

4401it [02:15, 32.41it/s]


In [5]:
meta_select_tag = lambda tag: lambda pr: any(tag in label.name for label in pr.labels)
select_merge_conflicts = meta_select_tag('2.status: merge conflict')
select_stale = meta_select_tag('2.status: stale')
select_automatic_upgrades = lambda pr: pr.user.login == 'r-ryantm'
select_automatic_backports = lambda pr: pr.user.login == 'github-actions[bot]'

In [7]:
current_merge_conflicts = list(filter(select_merge_conflicts, all_prs))
current_stale = list(filter(select_stale, all_prs))
automatic_upgrades = list(filter(select_automatic_upgrades, all_prs))
automatic_backports = list(filter(select_automatic_backports, all_prs))

In [8]:
import datetime
convert_timestamp = lambda s: datetime.datetime.strptime(s, '%a, %d %b %Y %H:%M:%S %Z')

[(pr.created_at, pr) for pr in sorted(current_merge_conflicts, key=lambda item: item.updated_at or item.created_at, reverse=True)]

[(datetime.datetime(2022, 12, 22, 2, 50, 53),
  PullRequest(title="pixelfed: init at 0.11.4, module", number=207194)),
 (datetime.datetime(2021, 12, 6, 10, 7, 49),
  PullRequest(title="osticket: add module", number=148959)),
 (datetime.datetime(2021, 12, 27, 16, 18, 4),
  PullRequest(title="rsync: Fix `enable*` options breaking build if disabled", number=152354)),
 (datetime.datetime(2022, 11, 19, 12, 48, 16),
  PullRequest(title="python3Packages.pandas: 1.4.4 -> 1.5.1", number=201898)),
 (datetime.datetime(2019, 10, 11, 14, 52, 10),
  PullRequest(title="VM test closure checks", number=70981)),
 (datetime.datetime(2022, 8, 15, 10, 10, 14),
  PullRequest(title="nixos/zfs_exporter: init", number=186777)),
 (datetime.datetime(2022, 12, 9, 15, 10, 52),
  PullRequest(title="nixos/minimal: remove environment.noXlibs true default", number=205318)),
 (datetime.datetime(2020, 7, 16, 23, 22, 51),
  PullRequest(title="Add kfp", number=93310)),
 (datetime.datetime(2022, 3, 22, 0, 24, 29),
  PullRe

In [9]:
from collections import Counter
Counter([pr.user.login for pr in all_prs if not pr.draft]).most_common()

[('r-ryantm', 585),
 ('github-actions[bot]', 114),
 ('SuperSandro2000', 36),
 ('amjoseph-nixpkgs', 35),
 ('fabaff', 26),
 ('wegank', 24),
 ('dit7ya', 21),
 ('jonringer', 20),
 ('alyssais', 19),
 ('Izorkin', 16),
 ('onny', 16),
 ('veprbl', 15),
 ('mkg20001', 15),
 ('risicle', 15),
 ('KAction', 15),
 ('emilytrau', 14),
 ('peperunas', 14),
 ('ncfavier', 13),
 ('IvarWithoutBones', 13),
 ('MatthewCroughan', 13),
 ('FRidh', 13),
 ('LeSuisse', 12),
 ('zhaofengli', 12),
 ('lopsided98', 12),
 ('dotlambda', 11),
 ('winterqt', 11),
 ('ShamrockLee', 11),
 ('matthiasbeyer', 11),
 ('viraptor', 11),
 ('ajs124', 10),
 ('roberth', 10),
 ('Flakebi', 10),
 ('infinisil', 10),
 ('nh2', 10),
 ('grahamc', 10),
 ('rhoriguchi', 9),
 ('minijackson', 9),
 ('foo-dogsquared', 9),
 ('Et7f3', 9),
 ('lourkeur', 9),
 ('sternenseemann', 9),
 ('milahu', 9),
 ('Synthetica9', 9),
 ('jtojnar', 8),
 ('mweinelt', 8),
 ('NickCao', 8),
 ('nagy', 8),
 ('xaverdh', 8),
 ('schnusch', 8),
 ('Enzime', 8),
 ('kilianar', 7),
 ('nrdxp'

In [39]:
from dataclasses import dataclass
import re

@dataclass
class Semver:
    major: int
    minor: int
    patch: int = 0
        
    @classmethod
    def from_str(cls, ver: str):
        assert is_semver(ver), "not a semantic version"
        return cls(*list(map(int, re.match('(?P<major>[0-9]+)\.(?P<minor>[0-9]+)\.(?P<patch>[0-9]+)', ver).groups())))
    
@dataclass
class SemverUpgradeNature:
    old_version: Semver
    new_version: Semver
    
    @property
    def major_level(self):
        return self.old_version.major != self.new_version.major
    
    @property
    def minor_level(self):
        return (not self.major_level) and (self.old_version.minor != self.new_version.minor)
    
    @property
    def patch_level(self):
        return (not self.major_level and not self.minor_level) and (self.old_version.patch != self.new_version.patch)

def is_semver(ver: str):
    try:
        groups = list(map(int, re.match('(?P<major>[0-9]+)\.(?P<minor>[0-9]+)\.(?P<patch>[0-9]+)', ver).groups()))
        return 2 <= len(groups) <= 3
    except:
        return False

def semver_nature(old_ver: str, new_ver: str):
    return SemverUpgradeNature(
        old_version=Semver.from_str(old_ver),
        new_version=Semver.from_str(new_ver)
    )
        
@dataclass
class UpgradeNature:
    ptags: list[str]
    pname: str
    old_version: str
    new_version: str
    
    @property
    def critical(self):
        return 'system' in self.ptags
    
    @property
    def semver(self):
        return is_semver(self.old_version) and is_semver(self.new_version)

    @property
    def patch_level(self):
        assert self.semver, "this upgrade do not use semantic versioning"
        return semver_nature(self.old_version, self.new_version).patch_level
    
    @property
    def minor_level(self):
        assert self.semver, "this upgrade do not use semantic versioning"
        return semver_nature(self.old_version, self.new_version).minor_level
    
    @property
    def major_level(self):
        assert self.semver, "this upgrade do not use semantic versioning"
        return semver_nature(self.old_version, self.new_version).major_level
    
    @property
    def semver_level(self):
        if self.patch_level: return 'patch'
        elif self.minor_level: return 'minor'
        elif self.major_level: return 'major'
        else: return 'unknown'
        
def analyze_upgrade(title):
    pname, version_update = title.split(':')
    old_version, new_version = version_update.strip().split(' -> ')
    
    return UpgradeNature(pname=pname, ptags=[], old_version=old_version, new_version=new_version)

In [45]:
from collections import defaultdict
partition = defaultdict(list)
for upgrade in automatic_upgrades:
    nature = analyze_upgrade(upgrade.title)
    if nature.semver:
        partition[nature.semver_level].append(upgrade)

In [49]:
len(partition['major']), len(partition['minor']), len(partition['patch'])

(49, 236, 268)

In [53]:
remaining = len(automatic_upgrades) - (len(partition['major']) + len(partition['minor']) + len(partition['patch']))
print(remaining)

137


In [69]:
reviews_for_patchlevel = [list(tqdm(pr.get_reviews())) for pr in partition['patch']]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.10it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.33it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.48it/s]
1it [00:00,  3.46it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.44it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.12it/s]
0it [00:00, ?it/s]
1it [00:00,  3.00it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.35it/s]
1it [00:00,  3.32it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?i

In [79]:
from IPython.core.display import HTML
[display(HTML('<a href="{link}">{link}</a>'.format(link=pr.html_url))) for pr in partition['patch'] if not pr.draft]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,